In [1]:
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM
import gradio as gr
import torch

/home/joshua/miniconda3/envs/llm-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#model_path = "/mnt/models/llm_storage/Phi-4-mini-instruct"
#model_path = "/home/joshua/llms/llama3.1_PC_Build/Llama-3.1-8B-Instruct"
model_path = "/mnt/models/llm_storage/Llama-3.2-3B-Instruct"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForCausalLM.from_pretrained(
    model_path, 
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
    device_map="cuda:0"
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.34it/s]


In [ ]:
system_context = {
    "role": "system",
        "content": (
            "You are SMARTConnect V2, a helpful and strategic AI assistant created to help small businesses discover ideal partnerships. "
            "You dont know there business or anbything about them until they tell you. "
            "Your goal is to help them find the best partnerships for their business. "
            "Ask relevant business questions first, and only generate partner recommendations and outreach emails once you have complete information."
        )
}

bot_context = [
    {
        "role": "assistant",
        "content": (
            "When the user initiates the conversation, begin by asking them to describe their business. "
            "Ask for what the bussiness is name and there primary purpose. "
            "Prompt them to explain what their company does and what kinds of products or services they offer. "
            "This will help lay the foundation for a personalized recommendation."
        )
    },
    {
        "role": "assistant",
        "content": (
            "Once the user has shared their business overview, follow up with more specific questions to fully understand their operation. "
            "Ask about their typical customer base (e.g. individuals, schools, small businesses), any accessories or services they provide "
            "(e.g. support, warranties, repairs), and whether they operate online, in-store, or both. "
            "These details will help you tailor your recommendations more precisely."
        )
    }
]


final_context = {
    "role": "assistant",
    "content": (
        "Once the user has provided complete details about their business, prepare to generate your recommendations. "
        "Clearly indicate to the user that you're now creating the output. "
        "List the top 3 most complementary business categories for partnerships based on the users information. "
        "For each complementary business list a reason, 2-3 strategic benefits, and a professional outreach email template. "
    )
}

def format_messages(messages):
    prompt = ""
    for msg in messages:
        role = msg["role"]
        if role == "system":
            prompt += f"<|system|>\n{msg['content']}\n"
        elif role == "user":
            prompt += f"<|user|>\n{msg['content']}\n"
        elif role == "assistant":
            prompt += f"<|assistant|>\n{msg['content']}\n"
    prompt += "<|assistant|>\n"
    return prompt

# outputs = pipeline(
#     prompt, 
#     max_new_tokens=2000,
#     length_penalty=1,
#     num_beams=6,
#     temperature=0.1,
#     top_p=0.8,
#     repetition_penalty=1,
#     )



In [ ]:
def generate_response(messages, max_new_tokens):
    prompt = format_messages(messages)
    #print(prompt)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        #top_p=0.95,
        #repetition_penalty=1.1,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("<|assistant|>")[-1]

def check_for_relevant_info(user_input, expected_question):
    relevance_check_context = [
        {
            "role": "system",
            "content": (
                "You are a strict assistant that determines whether a user's input is directly relevant to a specific question. "
                "You only respond with 'Yes' or 'No'. If the input very clearly addresses the question, say 'Yes'. "
                "If the input is vague, unrelated, or off-topic, say 'No'."
            )
        },
        {
            "role": "user",
            "content": f"Question: {expected_question}\nUser Input: {user_input}\nIs the user input relevant to the question?"
        }
    ]
    response = generate_response(relevance_check_context, max_new_tokens=100)
    print(f"User input: {user_input}")
    print(f"Expected question: {expected_question}")
    print(f"Relevance check response: {response}")
    return "yes" in response.strip().lower()

In [ ]:
load_context = [system_context] 
chat_stage = 0 


def smartconnect_chat(user_input):
    global chat_stage, load_context

    if user_input.strip().upper() == "RESET":
        load_context = [system_context]
        chat_stage = 0
        return "SMARTConnect V2: Memory cleared! Please type 'START' to begin again."

    if user_input.strip().upper() == "START" and chat_stage == 0:
        load_context.append({"role": "user", "content": bot_context[0]['content']})
        chat_response = generate_response(load_context, 400)
        load_context.append({"role": "assistant", "content": chat_response})
        chat_stage += 1
        return f"SMARTConnect V2: {chat_response}"

    elif 0 < chat_stage <= len(bot_context):
        expected_question = bot_context[chat_stage - 1]['content']
        if not check_for_relevant_info(user_input, expected_question):
            return "SMARTConnect V2: Let's stay on topic. Please rephrase your answer to better match the current question."

        load_context.append({"role": "user", "content": user_input})
        if chat_stage < len(bot_context):
            chat_response = generate_response(load_context, 400)
            load_context.append({"role": "assistant", "content": chat_response})
            chat_stage += 1
            return f"SMARTConnect V2: {chat_response}"

        else:
            load_context.append(final_context)
            final_response = generate_response(load_context, 2000)
            chat_stage += 1
            return f"SMARTConnect V2:\n\n{final_response}"

    else:
        return "SMARTConnect V2: Thank you! If you wish to restart, refresh the page."

gr.Interface(
    fn=smartconnect_chat,
    inputs=gr.Textbox(lines=2, placeholder="Type 'START' to begin"),
    outputs="text",
    title="SMARTConnect V2 - Business Partnership Finder",
    description="Say 'START' to begin the business discovery process. Type 'RESET' anytime to restart."
).launch(server_name="0.0.0.0", server_port=781, share=True)


Running on local URL:  http://0.0.0.0:7813


/home/joshua/miniconda3/envs/llm-env/lib/python3.10/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.37.2, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Running on public URL: https://7cdb3ccce495673253.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


User input: Oh Hi I currently run a PC building Business called MASTER RACERS
Expected question: When the user initiates the conversation, begin by asking them to describe their business. Ask for what the bussiness is name and there primary purpose. Prompt them to explain what their company does and what kinds of products or services they offer. This will help lay the foundation for a personalized recommendation.
Relevance check response: 
No. The user input is a statement about their business, but it doesn't address the question of laying the foundation for a personalized recommendation. It's a brief introduction to their business.


...

User input: POP
Expected question: Once the user has shared their business overview, follow up with more specific questions to fully understand their operation. Ask about their typical customer base (e.g. individuals, schools, small businesses), any accessories or services they provide (e.g. support, warranties, repairs), and whether they operate online, in-store, or both. These details will help you tailor your recommendations more precisely.
Relevance check response: 
No


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


User input: POP
Expected question: Once the user has shared their business overview, follow up with more specific questions to fully understand their operation. Ask about their typical customer base (e.g. individuals, schools, small businesses), any accessories or services they provide (e.g. support, warranties, repairs), and whether they operate online, in-store, or both. These details will help you tailor your recommendations more precisely.
Relevance check response: 
No


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


User input: POP
Expected question: Once the user has shared their business overview, follow up with more specific questions to fully understand their operation. Ask about their typical customer base (e.g. individuals, schools, small businesses), any accessories or services they provide (e.g. support, warranties, repairs), and whether they operate online, in-store, or both. These details will help you tailor your recommendations more precisely.
Relevance check response: 
No


In [ ]:
import json

# === Load flow JSON ===
with open("chat_flow.json") as f:
    chat_flow = json.load(f)

system_context = {
    "role": "system",
    "content": (
        "You are SMARTConnect V2, a helpful AI assistant. Your job is to guide users through a structured discovery conversation "
        "about their business. If a user asks something unrelated, answer it briefly and then return to the current business question."
    )
}

state_index = 0
chat_history = [system_context]

# === Format messages for LLaMA-style prompt ===
def format_prompt(messages):
    prompt = ""
    for msg in messages:
        prompt += f"<|{msg['role']}|>\n{msg['content']}\n"
    prompt += "<|assistant|>\n"
    return prompt

# === Inference ===
def generate_response(messages, max_new_tokens=400):
    prompt = format_prompt(messages)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded.split("<|assistant|>")[-1].strip()

def is_question_answered(user_input, validation_hint):
    check_prompt = (
        f"<|system|>\n"
        f"You are a strict validator. Only reply with 'yes' or 'no'.\n"
        f"Determine if this response answers the assistant’s current question.\n"
        f"The required answer should: {validation_hint}\n"
        f"User said: \"{user_input}\"\n"
        f"<|assistant|>\n"
        f"yes or no:"
    )

    inputs = tokenizer(check_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=2,
        do_sample=False,
        temperature=0.0,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True).lower().strip()
    return decoded.startswith("yes")


# === Gradio Chat Function ===
def smartconnect_chat(user_input):
    global state_index, chat_history

    if user_input.strip().upper() == "RESET":
        chat_history = [system_context]
        state_index = 0
        return "🔄 Chat reset. Let's begin. " + chat_flow[0]["prompt"]

    if state_index >= len(chat_flow):
        return "✅ All questions completed. Type 'RESET' to restart."

    # Get current question
    current_prompt = chat_flow[state_index]["prompt"]

    # Add user input to history
    chat_history.append({"role": "user", "content": user_input})

    # Add assistant's planned question (as reminder to LLM)
    chat_history.append({"role": "assistant", "content": current_prompt})

    # Generate response
    model_reply = generate_response(chat_history)

    validation_hint = chat_flow[state_index].get("validation_hint", "")
    if is_question_answered(user_input, validation_hint):
        state_index += 1
        if state_index < len(chat_flow):
            next_prompt = chat_flow[state_index]["prompt"]
            chat_history.append({"role": "assistant", "content": next_prompt})
            return model_reply + "\n\n**Next question:** " + next_prompt
        else:
            return model_reply + "\n\n✅ You're done! Type 'RESET' to restart."
    else:
        return model_reply + "\n\n🔁 Please try answering the question more directly."


# === Launch Gradio ===
gr.Interface(
    fn=smartconnect_chat,
    inputs=gr.Textbox(lines=2, placeholder="Say hi or type 'RESET'"),
    outputs="text",
    title="SMARTConnect V2 – Business Discovery Assistant",
    description="Guided by JSON, answered by your local LLaMA model. Type 'RESET' anytime to restart.",
).launch(server_name="0.0.0.0", server_port=7869, share=True)


Running on local URL:  http://0.0.0.0:7869


/home/joshua/miniconda3/envs/llm-env/lib/python3.10/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.37.2, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Running on public URL: https://9905d7fe06cc7226fc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
/home/joshua/miniconda3/envs/llm-env/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/joshua/miniconda3/envs/llm-env/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for 

: 

In [ ]:
# load_context = [system_context]

# print("SMARTConnect V2: This is a business connection tool for Small Tech Businesses, say START to begin")
# while user_input != "START":
#     user_input = input("You: ")

# print("SMARTConnect V2: Alright lets begin!")
# for i in range(len(bot_context)):
#     load_context.append(bot_context[i])
#     chat_response = generate_response(load_context, 400)
#     #print(f"\nDebug - Loading Context:\n{i}\n")
#     print(f"\nSMARTConnect V2:\n{chat_response}\n")
    
#     user_input = input("You: ")
#     load_context.append({"role": "user", "content": user_input})

# print("\nSMARTConnect V2 is generating your tailored recommendations...\n")
# load_context.append(final_context)
# final_response = generate_response(load_context, 2000)
# print(f"\nSMARTConnect V2:\n{final_response}\n")

# OUTPUT EXAMPLE
# SMARTConnect V2:

# Based on our conversation, I recommend the following top 3 business categories for partnerships:

# **1. Local Food Suppliers**

# Reason: Aligning with your focus on quality and sustainability, partnering with local food suppliers can ensure consistent freshness and reduce carbon footprint.

# Strategic Benefits:

# * Secure access to high-quality ingredients
# * Support local businesses and contribute to community development
# * Enhance your brand's reputation for sustainability

# Outreach Email Template:

# Subject: Exploring Partnership Opportunities for Fresh Ingredients

# Dear [Supplier's Name],

# I hope this email finds you well. As a cooking business focused on delivering quality meals, I'm excited to explore potential partnership opportunities with your local food supplier company. Our shared commitment to sustainability and quality aligns perfectly with your business model.

# Would you be open to discussing how we can work together to source fresh, locally-sourced ingredients for our customers? I believe our collaboration could bring numerous benefits to both our businesses.

# Looking forward to hearing from you.

# Best regards,
# [Your Name]

# **2. Kitchen Appliance Brands**

# Reason: Partnering with kitchen appliance brands can enhance your business's reputation for quality and provide customers with integrated solutions.

# Strategic Benefits:

# * Offer bundled solutions with appliances and cooking services
# * Enhance your brand's reputation for quality and expertise
# * Access to innovative appliances and cooking technologies

# Outreach Email Template:

# Subject: Partnership Opportunities for Integrated Cooking Solutions

# Dear [Appliance Brand's Name],

# I'm reaching out to explore potential partnership opportunities between our cooking business and your kitchen appliance brand. Our shared passion for quality and innovation aligns perfectly with your business model.

# Would you be interested in discussing how we can collaborate to offer bundled solutions with our cooking services and your appliances? This could bring significant benefits to both our businesses, including enhanced customer satisfaction and increased brand visibility.

# Looking forward to hearing from you.

# Best regards,
# [Your Name]

# **3. Cooking Schools or Health and Wellness Brands**

# Reason: Partnering with cooking schools or health and wellness brands can expand your customer base, enhance your brand's reputation, and offer new services.

# Strategic Benefits:

# * Access new customer segments and expand your business
# * Collaborate on cooking classes, workshops, or events
# * Enhance your brand's reputation for expertise and commitment to healthy eating

# Outreach Email Template:

# Subject: Exploring Partnership Opportunities for Cooking Classes and Events

# Dear [Cooking School's Name],

# I hope this email finds you well. As a cooking business focused on delivering quality meals, I'm excited to explore potential partnership opportunities with your cooking school. Our shared passion for cooking and healthy eating aligns perfectly with your business model.

# Would you be interested in discussing how we can collaborate on cooking classes, workshops, or events that showcase our expertise and your school's offerings? This could bring numerous benefits to both our businesses, including expanded customer reach and enhanced brand visibility.

# Looking forward to hearing from you.

# Best regards,
# [Your Name]

# Which of these partnerships resonates with you the most, or would you like to explore other options?

SMARTConnect V2: This is a business connection tool for Small Tech Businesses, say START to begin


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


SMARTConnect V2: Alright lets begin!

SMARTConnect V2:
Another question to help me better understand your business: What industries or sectors is your business primarily operating in? For example, is it healthcare, finance, retail, technology, or something else? This will help me provide more targeted and relevant partnership suggestions.



NameError: name 'display_in_browser' is not defined